In [ ]:
# SETUP

# copied stats 13th-15th
# gsutil -m cp -R gs://ext-kiwi-excl-stat/stats/y=2020/m=06/d=13/* \
                 #gs://ext-kiwi-excl-dev-stat/stats/y=2020/m=06/d=13/
    
# copied data 13th-15th
gsutil -m cp -R gs://ext-kiwi-excl-data/data/y=2020/m=06/d=13/hr=00/part-00000* \
            gs://ext-kiwi-excl-dev-data/data/y=2020/m=06/d=13/hr=00/   

# update forex else X5b returns nothing
gsutil cp gs://fxrates/to/data.csv gs://bq-generic-dev/fxrates/data.csv
        

In [52]:
from datetime import datetime, timedelta

In [53]:
def getTS():
    dt = datetime.now()
    # [:-1] used otherwise the integer is greater than BQ will accept
    return [ int(dt.strftime("%Y%m%d%H%M%S%f")[:-1]) , dt.strftime("%Y-%m-%d %H:%M:%S.%f UTC") ]

In [68]:
l = getTS()
print(l)
print(type(l[0]))

[2020061522094746102, '2020-06-15 22:09:47.461021 UTC']
<class 'int'>


In [6]:
ds_nodash = '20200601'
print( '%stat/stats/y={}/m={}/d={}/hr%.csv'.format(ds_nodash[:4],ds_nodash[4:6],ds_nodash[6:]) )

# y={{ ds_nodash }}[:4]/m={{ ds_nodash }}[4:6]/d={{ ds_nodash }}[6:]

%stat/stats/y=2020/m=06/d=01%.csv


In [70]:
# gccProject = '{{ var.value.gccProject }}'
gccProject = 'skytra-benchmark-devandtest'

dictX2PassValue = {'skytra-benchmark-devandtest': '24',
                   'skytra-benchmark-uat': '2',
                   'd-dat-digitalaircrafttransport': '24'}

passVal = dictX2PassValue[gccProject]

sql = f'select FileCount = {passVal} from tb where ExecDate = ' + '{{ ds_nodash }} and DATE(ts) = CURRENT_DATE()'
print(sql)


select FileCount = 24 from tb where ExecDate = {{ ds_nodash }} and DATE(ts) = CURRENT_DATE()


In [13]:
#af env 
# varsBucketEnvString
# gccProject
# ProcessingLocation

In [73]:
ds_no_dot = ds.replace('.','')
print(ds_no_dot)

TypeError: 'str' object cannot be interpreted as an integer

In [66]:
quer_base = 'bq query --location=EU --use_legacy_sql=false "' 
#cmd_log_sfx = " '{{ ti.xcom_pull('Get-ParamsX7')[0][2] }}', '{{ macros.ds_add(ti.xcom_pull('Get-ParamsX7')[0][2], 6) }}', CURRENT_TIMESTAMP );" + '"'
quer_log_sfx = " '{{ ds }}' as dateFrom, '{{ ds }}' as dateTo, 0 as bq_rc, 0 as bq_bp, 0 as bq_bb, '{{ ts }}' as ts_end"

In [20]:
# test f'' with jinja
# FAIL
tbl = 'airflow.af_check_X2'
passVal=24
'select FileCount={} from {} where ExecDate = {{ ds_nodash }} and DATE(ts)=CURRENT_DATE()'.format(passVal,tbl)

'select FileCount=24 from airflow.af_check_X2 where ExecDate = { ds_nodash } and DATE(ts)=CURRENT_DATE()'

In [24]:
sqlCheck = 'select {{ ds_nodash }} as ExecDate, ' + """
    sum(if(metric_name='parsedKiwiItinerariesAcc', metric_value,0)) as KiwiItinAcc,
    sum(if(metric_name='validAirbusItinerariesAcc', metric_value,0)) as AirbusItinAcc,
    cast(count(*)/2 as int64) as FileCount,
    CURRENT_TIMESTAMP as ts
    from stat.kiwi_stats 
    where 
    metric_name in('parsedKiwiItinerariesAcc', 'validAirbusItinerariesAcc') AND 
    _FILE_NAME like '%stat/stats/y={}/m={}/d={}/hr%.csv'
    """.format(ds_nodash[:4],ds_nodash[4:6],ds_nodash[6:])

print(sqlCheck)

select {{ ds_nodash }} as ExecDate, 
    sum(if(metric_name='parsedKiwiItinerariesAcc', metric_value,0)) as KiwiItinAcc,
    sum(if(metric_name='validAirbusItinerariesAcc', metric_value,0)) as AirbusItinAcc,
    cast(count(*)/2 as int64) as FileCount,
    CURRENT_TIMESTAMP as ts
    from stat.kiwi_stats 
    where 
    metric_name in('parsedKiwiItinerariesAcc', 'validAirbusItinerariesAcc') AND 
    _FILE_NAME like '%stat/stats/y=2020/m=06/d=01/hr%.csv'
    


In [30]:

din = datetime(2020, 6, 13)
dout = din.strftime('y=%Y/m=%m/d=%d')
print(din)
print(dout)

# {{ macros.ds_format(ds,'%Y-%m-%d','y=%Y/m=%m/d=%d') }}

2020-06-13 00:00:00
y=2020/m=06/d=13


In [3]:


####### JINJA #############################################

from jinja2 import Template


In [12]:
template = Template('Hello {{ name }}!')
template.render(name='John Doe')

'Hello John Doe!'

In [16]:

for i in range(24):  
    #template = Template('Hello {{ name }}!')
    #print(template.render(name='John Doe'))
    
    #print(str(i))
    template = Template('Hello {{ name }} {{ int }}!')
    print(template.render({'name':'John Doe', 'int':i}))

Hello John Doe 0!
Hello John Doe 1!


In [22]:
#load_base + '--replace --source_format=PARQUET kiwi.X2_{{ ds_nodash[4:] }}' + f' "gs://ext-kiwi-excl-data/data/' + '{{ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") }}/*.gz.parquet"'

#--replace then append
template = Template('--source_format=PARQUET kiwi.X2_{{ ds_nodash[4:] }} "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"')

# old pop logic in orig Kiwi loads

for hr in range(1,5):
    print(template.render({'ds_nodash':'20201216', 'ds':'2020-12-16'})) 
    op = template.render({'ds_nodash':'20201216', 'ds':'2020-12-16'})


--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"
--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"
--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"
--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"


In [28]:
#from af\dags\dag_Kiwi_2020_3x.py

#will need to make t3

max_hr = 24
aX2 = []
t1 = ['t1']
t3 = ['t3']

for hr in range(1, max_hr + 1):
    
    tX2 = hr
    
    aX2.append(tX2)
    
#t1 >> aX2 >> t3
aX2

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [27]:
str(unlist(aX2))

NameError: name 'unlist' is not defined

In [ ]:
template = Template('--source_format=PARQUET kiwi.X2_{{ ds_nodash[4:] }} "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"')

# old pop logic in orig Kiwi loads

for hr in range(1,5):
    print(template.render

In [4]:
tpay = """{% set payment_methods = ["bank_transfer", "credit_card", "gift_card"] %}

select
    order_id,
    {% for payment_method in payment_methods %}
    sum(case when payment_method = '{{payment_method}}' then amount end) as {{payment_method}}_amount,
    {% endfor %}
    sum(amount) as total_amount
from app_data.payments
group by 1"""

j_tpay = template = Template(tpay)

j_tpay.render()

"\n\nselect\n    order_id,\n    \n    sum(case when payment_method = 'bank_transfer' then amount end) as bank_transfer_amount,\n    \n    sum(case when payment_method = 'credit_card' then amount end) as credit_card_amount,\n    \n    sum(case when payment_method = 'gift_card' then amount end) as gift_card_amount,\n    \n    sum(amount) as total_amount\nfrom app_data.payments\ngroup by 1"

In [24]:
hour_list = []
for i in range(1,25):
    hour_list.append(f'{i:02d}')
hour_list

['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24']

In [25]:
#--replace then append


#tPARQUET = '{% for i in range(1,25) %} \'--source_format=PARQUET kiwi.X2_{{ ds_nodash[4:] }} "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d") /*.gz.parquet"\' {% endfor %}'
tPARQUET = '''{% for i in ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'] %}
--source_format=PARQUET kiwi.X2_{{ ds_nodash[4:] }} "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr={{ i }}")/*.gz.parquet"
{% endfor %}'''


jPARQUET = Template(tPARQUET)

jPARQUET.render({'ds_nodash':'20201216'})

'\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=01") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=02") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=03") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=04") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=05") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=06") /*.gz.parquet"\n\n\n--source_format=PARQUET kiwi.X2_1216 "gs://ext-kiwi-excl-data/data/ macros.ds_format(ds,"%Y-%m-%d","y=%Y/m=%m/d=%d/hr=07") /*.gz.parquet"\n

In [ ]:

####### JINJA AIRFLOW User Defined Macros#############################################

def compute_next_execution_date(dag, execution_date):
    return dag.following_schedule(execution_date)

dag = DAG(
    'simple',
    schedule_interval='0 21 * * *',
    user_defined_macros={
        'next_execution_date': compute_next_execution_date,
    },
)

task = BashOperator(
    task_id='bash_op',
    bash_command='echo "{{ next_execution_date(dag, execution_date) }}"',
    dag=dag,
)

In [50]:
w_dis = 'WRITE_TRUNCATE'
#w_dis = 'other'
dest = 'kiwi.X4$20200615'
#dest = 'kiwi.X4'
if w_dis == 'WRITE_APPEND' or (w_dis == 'WRITE_TRUNCATE' and '$' in dest):
    allow_new = ['ALLOW_FIELD_ADDITION']
else:
    allow_new = []
        
print(allow_new)

['ALLOW_FIELD_ADDITION']


In [75]:
# op_gen SECTION

from datetime import datetime, timedelta
#from airflow.operators.bash_operator import BashOperator
#from airflow.contrib.operators.bigquery_operator import BigQueryOperator


def mk_BQO(tid, sqlFile, dest, c_dis, w_dis, redo, redo_delay_mins, DEBUG_MODE, time_part_col = None):
    time_part = (None if time_part_col == None else {'type':'day', 'field':time_part_col})
    if DEBUG_MODE:
        tid = tid + '_DEBUG'
        dest = 'airflow_debug.' + dest.replace('.','_')
        c_dis = 'CREATE_IF_NEEDED'
        #sqlFile = sqlFile # + ' LIMIT 1000' this did not work with jinja templating

    # Schema update options should only be specified with WRITE_APPEND disposition, or with WRITE_TRUNCATE disposition on a table partition
    if w_dis == 'WRITE_APPEND' or (w_dis == 'WRITE_TRUNCATE' and '$' in dest):
        allow_new = ['ALLOW_FIELD_ADDITION']
    else:
        allow_new = []
    
    """
    op = BigQueryOperator(
        task_id = tid,
        sql = sqlFile,
        destination_dataset_table = dest,
        create_disposition = c_dis,
        write_disposition = w_dis,
        schema_update_options = allow_new,
        allow_large_results = True,
        use_legacy_sql = False,
        retries = redo,  
        retry_delay = timedelta(minutes = redo_delay_mins)
    )
    return op
    """

In [80]:
# test can push in var to dict value
col = 'search_date'
time_partitioning={'type':'day','field':col}
print(time_partitioning['field'])

search_date


In [83]:
time_part_col = 'search_date'
time_part = (None if time_part_col == None else {'type':'day', 'field':time_part_col})
print(time_part)

{'type': 'day', 'field': 'search_date'}


In [3]:
ds_nodash = '20200909'
ds_nodash[4:]

'0909'